# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298388877121                   -0.85    5.2         
  2   -8.300191348018       -2.74       -1.26    1.0    102ms
  3   -8.300430255059       -3.62       -1.89    2.1    120ms
  4   -8.300461618642       -4.50       -2.74    2.8    186ms
  5   -8.300463771235       -5.67       -3.05    2.4    133ms
  6   -8.300464194414       -6.37       -3.23    9.1    214ms
  7   -8.300464427638       -6.63       -3.39    1.8    123ms
  8   -8.300464549018       -6.92       -3.54    2.9    139ms
  9   -8.300464621352       -7.14       -3.76    4.4    187ms
 10   -8.300464634828       -7.87       -3.91    1.9    138ms
 11   -8.300464643434       -8.07       -4.30    2.6    369ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64164429995                   -0.70    6.8         
  2   -16.67868528328       -1.43       -1.14    1.5    323ms
  3   -16.67920071586       -3.29       -1.88    2.0    676ms
  4   -16.67927249460       -4.14       -2.73    3.5    331ms
  5   -16.67928557354       -4.88       -3.09    4.8    412ms
  6   -16.67928619778       -6.20       -3.51    1.9    279ms
  7   -16.67928621604       -7.74       -4.02    2.0    297ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32480502832                   -0.56    6.8         
  2   -33.33251656091       -2.11       -1.00    1.8    1.01s
  3   -33.33409946309       -2.80       -1.73    4.0    1.22s
  4   -33.33568363756       -2.80       -2.37    3.2    1.19s
  5   -33.33692474701       -2.91       -2.69   13.2    1.77s
  6   -33.33694265025       -4.75       -3.23    4.6    1.27s
  7   -33.33694366529       -5.99       -3.82    3.1    1.28s
  8   -33.33694376234       -7.01       -4.26    4.0    1.27s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298326325775                   -0.85    5.0         
  2   -8.300245930421       -2.72       -1.58    1.0   85.9ms
  3   -8.300390735093       -3.84       -2.35    2.2    112ms
  4   -8.300319374904   +   -4.15       -2.17    3.4    167ms
  5   -8.300463773439       -3.84       -3.44    1.0   84.6ms
  6   -8.300464418764       -6.19       -3.60    5.6    190ms
  7   -8.300464625042       -6.69       -4.04    1.8    105ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32527472013                   -0.56    8.4         
  2   -33.31681696846   +   -2.07       -1.27    1.5    860ms
  3   -15.66598430952   +    1.25       -0.44    6.5    1.77s
  4   -33.32570750095        1.25       -1.94    5.5    1.55s
  5   -33.31509989090   +   -1.97       -1.60    3.1    1.40s
  6   -33.31688673339       -2.75       -1.83    3.2    1.23s
  7   -33.15281967155   +   -0.78       -1.43    4.9    1.35s
  8   -33.33610712283       -0.74       -2.50    3.6    1.18s
  9   -33.33622133779       -3.94       -2.58    3.0    1.18s
 10   -33.33679285520       -3.24       -2.63    2.5    1.10s
 11   -33.33684116318       -4.32       -3.01    1.9    984ms
 12   -33.33693991287       -4.01      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.